In [1]:
from PIL import Image
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image 
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from glob import glob
import matplotlib.pyplot as plt

In [2]:
IMAGE_SIZE = [224,224]
train_path=r"C:\Users\Jaideep Naroo\Downloads\archive(1)\chest_xray\train"
valid_path=r"C:\Users\Jaideep Naroo\Downloads\archive(1)\chest_xray\test"


In [3]:
#Using Imagenet weight 
#Import the vgg vgg16 lib as shown below and add preprocessing layer to the front of VGG
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top= False)

In [4]:
# We will not train the existing weigths because it will change the weigths that have already been captured
for layer in vgg.layers:
    layer.trainable = False

In [5]:
# We need to know how many classes does our train folder have so we know what outputs we will be getting 
folders = glob(r"C:\Users\Jaideep Naroo\Downloads\archive(1)\chest_xray\train\*")

In [6]:
# We will make one Flatten Layer
x= Flatten()(vgg.output)

In [7]:
# we will add our last layer
prediction = Dense(len(folders),activation='softmax')(x) 

# create our model object
model=Model(inputs=vgg.input, outputs=prediction)

In [8]:
#To view the structure of our model
model.summary()
# 

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
#Model compilation using adam optimizer and accuracy
model.compile(
  loss='categorical_crossentropy',
  optimizer = 'adam',
  metrics=['accuracy']
)

In [10]:
#using the image data generator to import the images from the dataset
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [11]:
# We will be now inserting some images  via flow from directory
# Make sure u provide the same target size as initialized for the image size
training_set = train_datagen.flow_from_directory(r"C:\Users\Jaideep Naroo\Downloads\archive(1)\chest_xray\train",
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 5216 images belonging to 2 classes.


In [12]:
test_set = test_datagen.flow_from_directory(r"C:\Users\Jaideep Naroo\Downloads\archive(1)\chest_xray\test",
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 624 images belonging to 2 classes.


In [13]:
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
 )

Epoch 1/5
163/163 [==============================] - 1019s 6s/step - loss: 0.2299 - accuracy: 0.9124 - val_loss: 0.3187 - val_accuracy: 0.8926
Epoch 2/5
163/163 [==============================] - 1853s 11s/step - loss: 0.1132 - accuracy: 0.9561 - val_loss: 0.2295 - val_accuracy: 0.9151
Epoch 3/5
163/163 [==============================] - 1041s 6s/step - loss: 0.0884 - accuracy: 0.9668 - val_loss: 0.2423 - val_accuracy: 0.9119
Epoch 4/5
163/163 [==============================] - 1180s 7s/step - loss: 0.0939 - accuracy: 0.9657 - val_loss: 0.6078 - val_accuracy: 0.8365
Epoch 5/5
163/163 [==============================] - 1060s 7s/step - loss: 0.0852 - accuracy: 0.9664 - val_loss: 0.2431 - val_accuracy: 0.9231


In [14]:
# save it as a h5 file

import tensorflow as tf

from keras.models import load_model

model.save('model_vgg16.h5')



In [32]:
from keras.models import load_model
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
import numpy as np

In [34]:
model = load_model('model_vgg16.h5')
img = tf.keras.utils.load_img(r"C:\Users\Jaideep Naroo\Downloads\archive(1)\chest_xray\train\PNEUMONIA\person9_bacteria_40.jpeg", target_size=(224, 224))
x = tf.keras.utils.img_to_array(img)

In [35]:
x= np.expand_dims(x, axis=0)
img_data = preprocess_input(x)
classes = model.predict(img_data)

1/1 [==============================] - 0s 445ms/step


In [53]:


acc=model.evaluate(training_set)[1]
print(f"The accuracy of model is {acc*100} %")



163/163 [==============================] - 1063s 7s/step - loss: 0.0752 - accuracy: 0.9724
The accuracy of model is 97.23926186561584 %
